# Estimativa de pontos de estória usando aprendizado profundo

Utilização de aprendizado profundo por redes neurais.

In [6]:
import numpy as np
import pandas as pd

import spacy
import keras

from collections import defaultdict

nlp = spacy.load('en', disable=['parser', 'tagger', 'ner'])
#tokenizar = spacy.tokenizer.Tokenizer(nlp.vocab)

Using Theano backend.


## Divisão do conjunto de dados

Treino, validação e teste.

In [4]:
def ler_csv_texto_tokenizado(caminho_arquivo, tokenizador):
    tabela_dados = pd.read_csv(caminho_arquivo)
    titulos = tabela_dados.values[:, 1].astype('str')
    separadores = np.full(len(titulos), '. ')
    descricao = tabela_dados.values[:, 2].astype('str')
    juntos = [a + b + c for a, b, c in zip(titulos, separadores, descricao)]
    tokenizados = [[w.text.strip().lower() for w in tokenizador(frase) if not w.text.isspace()] for frase in juntos]
    return tokenizados

def gerar_dicionario(lista_tokens, vocabulario_maximo=100000):
    contagem = defaultdict(int)
    for tokens in lista_tokens:
        for token in tokens:
            contagem[token] += 1
    ordenadas = sorted(contagem.items(), key=lambda x: x[1], reverse=True)
    IDs = {}
    for i in range(min(vocabulario_maximo, len(ordenadas))): # EOF = 0, ID_desconhecido = 1
        IDs[token] = i + 2
    return IDs

def converte_tokens(lista_tokens, dic, fracao_valida = 0.1):
    lista_ids = [[(dic.get(tk) or 0) for tk in tks] for tks in lista_tokens]
    quant_valida = int(fracao_valida * len(lista_ids))
    return lista_ids[:-quant_valida], lista_ids[-quant_valida:]

In [5]:
a = ler_csv_texto_tokenizado('dados/pretreino/moodle.csv', nlp.tokenizer)
dic = gerar_dicionario(a)
treino, validacao = converte_tokens(a, dic)

## Pretreinamento

In [ ]:
class EstimativaConstrastivaRuidosa(keras.engine.topology.Layer):
    def __init__(self, init='glorot_uniform', comprimento=10,
                 dimensao_entrada=None, vocabulario=None, ruidos = 25, distribuicao_ruidos=[0.5, 0.5], **kwargs):
        self.init = init
        self.comprimento = comprimento
        self.vocabulario = vocabulario
        self.ruidos = ruidos
        self.distribuicao_ruidos = theano.shared(numpy.array(distribuicao_ruidos).astype(config.floatX)) #IMP
        self.gerador_aleatorio = RS.RandomStreams(seed=SEED) #IMP
        kwargs['input_shape'] = (self.dimensao_entrada, )
        super(EstimativaConstrastivaRuidosa, self).__init__(**kwargs)

    def build(self, dims_entrada):
        self.W = self.add_weight(name='{}_W'.format(self.name), shape=(self.vocabulario, self.dimensao_entrada), initializer=self.init, trainable=True)
        self.b = self.add_weight(name='{}_b'.format(self.name), shape=(self.vocabulario, )              , initializer=self.init, trainable=True)
        super(EstimativaConstrastivaRuidosa, self).build(dims_entrada)

    def compute_output_shape(self, dims_entrada):
        return (None, self.comprimento, self.ruidos + 1)

    def call(self, inputs, mask=None):
        contexto = inputs[0] #shape: amostras * passos * dim
        proxima_palavra = inputs[1] #shape: amostras * passos

        amostras, passos = proxima_palavra.shape
        dim_saida = self.ruidos + 1

        noise_w = self.gerador_aleatorio.choice(size=(amostras, passos, self.ruidos), a=self.distribuicao_ruidos.shape[0], p=self.distribuicao_ruidos)
        proxima_palavra = proxima_palavra.flatten().reshape([amostras, passos, 1])
        proxima_palavra = tensor.concatenate([proxima_palavra, noise_w], axis=-1) #IMP shape: amostras * passos * dim_saida

        W_ = self.W[proxima_palavra.flatten()].flatten().reshape([amostras, passos, dim_saida, self.dimensao_entrada])
        b_ = self.b[proxima_palavra.flatten()].reshape([amostras, passos, dim_saida])

        s_theta = (contexto[:, :, None, :] * W_).sum(axis=-1) + b_ # dims: amostras * passos * dim_saida
        noiseP = self.distribuicao_ruidos[proxima_palavra.flatten()].reshape([amostras, passos, dim_saida])
        noise_score = K.log(self.ruidos * noiseP) #log(k * distribuicao_ruidos(w))

        return activations.sigmoid(s_theta - noise_score) # dims: amostras, passos, dim_saida

In [7]:
def modelo_pretreinamento(dicionario):
    quant_vocab = len(dicionario)
    

## Treinamento

## Avaliação